In [4]:
# Librerías necesarias
import pandas as pd

# Ruta al archivo 
ruta = '../data/raw_data/fonasa_2024.csv'

# Leer el CSV (ajusta el nombre según tu archivo)
df = pd.read_csv(ruta,encoding='latin1')

# Ver las primeras filas
print(df.head())

   MES_INFORMACION                     REGIÓN  COMUNA    SEXO    EDAD_TRAMO  \
0           202412  Metropolitana De Santiago   Macul   Mujer  50 a 54 años   
1           202412  Metropolitana De Santiago   Macul  Hombre  30 a 34 años   
2           202412                 Del Bíobío   Penco   Mujer  45 a 49 años   
3           202412                 Del Bíobío  Antuco  Hombre  65 a 69 años   
4           202412            De La Araucanía  Temuco   Mujer  25 a 29 años   

  NACIONALIDAD TITULAR_CARGA TRAMO  BENEFICIARIOS  
0      Chilena       Titular     C            570  
1      Chilena       Titular     D           1810  
2      Chilena       Titular     B            518  
3      Chilena       Titular     B             68  
4      Chilena       Titular     D           2414  


In [23]:
## Filtramos para eliminar valores nulos
df = df[df["COMUNA"] != "Desconocida"]

In [13]:
df_macul = df[df["COMUNA"] == "Macul"]

In [25]:
df_grouped = (
    df.groupby(["REGIÓN", "COMUNA", "NACIONALIDAD"])["BENEFICIARIOS"]
      .sum()
      .reset_index()
)
print(df_grouped.head(10))

           REGIÓN       COMUNA NACIONALIDAD  BENEFICIARIOS
0  De Antofagasta  Antofagasta      Chilena         263334
1  De Antofagasta  Antofagasta   Extranjera          86784
2  De Antofagasta       Calama      Chilena         112470
3  De Antofagasta       Calama   Extranjera          41553
4  De Antofagasta  María Elena      Chilena           1915
5  De Antofagasta  María Elena   Extranjera            739
6  De Antofagasta   Mejillones      Chilena           4917
7  De Antofagasta   Mejillones   Extranjera           4179
8  De Antofagasta      Ollagüe      Chilena             32
9  De Antofagasta      Ollagüe   Extranjera             33


In [19]:
print(df["REGIÓN"].unique())

['Metropolitana De Santiago' 'Del Bíobío' 'De La Araucanía' 'De Coquimbo'
 'De Tarapacá' 'De Valparaíso' 'De Los Lagos' 'De Antofagasta'
 "Del Libertador General Bernardo O'Higgins" 'Del Maule' 'De Ñuble'
 'De Los Ríos' 'Desconocida' 'De Magallanes Y De La Antártica Chilena'
 'De Atacama' 'De Aysén Del Gral. C. Ibáñez Del Campo'
 'De Arica Y Parinacota' nan]


In [27]:
df_grouped[df_grouped["REGIÓN"]=="Metropolitana De Santiago"]

,REGIÓN,COMUNA,NACIONALIDAD,BENEFICIARIOS
588,Metropolitana De Santiago,Alhué,Chilena,7483
589,Metropolitana De Santiago,Alhué,Extranjera,1675
590,Metropolitana De Santiago,Buin,Chilena,94304
591,Metropolitana De Santiago,Buin,Extranjera,6505
592,Metropolitana De Santiago,Calera De Tango,Chilena,19546
...,...,...,...,...
687,Metropolitana De Santiago,Tiltil,Extranjera,1576
688,Metropolitana De Santiago,Vitacura,Chilena,16761
689,Metropolitana De Santiago,Vitacura,Extranjera,4868
690,Metropolitana De Santiago,Ñuñoa,Chilena,96673


In [33]:
# Pivotar nacionalidad a columnas
df_pivot = df_grouped.pivot_table(
    index=["REGIÓN", "COMUNA"],
    columns="NACIONALIDAD",
    values="BENEFICIARIOS",
    aggfunc="sum",
    fill_value=0
).reset_index()

# Renombrar columnas para que queden más claras
df_pivot = df_pivot.rename(columns={
    "Chilena": "BENEFICIARIOS_CHILENOS",
    "Extranjera": "BENEFICIARIOS_EXTRANJEROS"
})

df_pivot.columns.name = None
df_pivot.head(10)

,REGIÓN,COMUNA,BENEFICIARIOS_CHILENOS,BENEFICIARIOS_EXTRANJEROS
0,De Antofagasta,Antofagasta,263334,86784
1,De Antofagasta,Calama,112470,41553
2,De Antofagasta,María Elena,1915,739
3,De Antofagasta,Mejillones,4917,4179
4,De Antofagasta,Ollagüe,32,33
5,De Antofagasta,San Pedro De Atacama,8787,3845
6,De Antofagasta,Sierra Gorda,385,532
7,De Antofagasta,Taltal,6158,913
8,De Antofagasta,Tocopilla,14630,2542
9,De Arica Y Parinacota,Arica,193880,47089


In [36]:
# Calcular total de beneficiarios
df_pivot["TOTAL_BENEFICIARIOS"] = (
    df_pivot["BENEFICIARIOS_CHILENOS"] + df_pivot["BENEFICIARIOS_EXTRANJEROS"]
)

# Calcular porcentaje de extranjeros
df_pivot["PORCENTAJE_EXTRANJEROS"] = (
    df_pivot["BENEFICIARIOS_EXTRANJEROS"] / df_pivot["TOTAL_BENEFICIARIOS"] * 100
)

# redondear a 2 decimales
df_pivot["PORCENTAJE_EXTRANJEROS"] = df_pivot["PORCENTAJE_EXTRANJEROS"].round(2)

df_pivot.head()

,REGIÓN,COMUNA,BENEFICIARIOS_CHILENOS,BENEFICIARIOS_EXTRANJEROS,TOTAL_BENEFICIARIOS,PORCENTAJE_EXTRANJEROS
0,De Antofagasta,Antofagasta,263334,86784,350118,24.79
1,De Antofagasta,Calama,112470,41553,154023,26.98
2,De Antofagasta,María Elena,1915,739,2654,27.84
3,De Antofagasta,Mejillones,4917,4179,9096,45.94
4,De Antofagasta,Ollagüe,32,33,65,50.77


In [37]:
df_pivot[df_pivot["PORCENTAJE_EXTRANJEROS"]> 50]

,REGIÓN,COMUNA,BENEFICIARIOS_CHILENOS,BENEFICIARIOS_EXTRANJEROS,TOTAL_BENEFICIARIOS,PORCENTAJE_EXTRANJEROS
4,De Antofagasta,Ollagüe,32,33,65,50.77
6,De Antofagasta,Sierra Gorda,385,532,917,58.02
10,De Arica Y Parinacota,Camarones,68,131,199,65.83
11,De Arica Y Parinacota,General Lagos,102,151,253,59.68
306,Metropolitana De Santiago,Independencia,57129,58360,115489,50.53


In [39]:
df_pivot.to_csv('../data/processed_data/fonasa.csv', index=False)